### Importing libraries

In [ ]:
library(funFEM)
library(tidyverse)
library(ggmap)
library(gridExtra)
library(RColorBrewer)
library(ggthemes)
data(velib)

### Preparing data

In [ ]:
x <- as.matrix(velib$data)
colnames(x) <- 1:ncol(x)
rownames(x) <- velib$names

bikes <- tibble(velib$data)
bikes$long <- velib$position$longitude
bikes$lat <- velib$position$latitude
bikes$hill <- as.factor(velib$bonus)
bikes$name <- velib$name

fig <- function(width, heigth){
     options(repr.plot.width = width, repr.plot.height = heigth)
}

name_func <- function(name) {
    paste('T', as.numeric(substring(name, first=2))-13, sep='')
}

bikes <- bikes %>% 
    select(!V1:V13) %>%
    rename_with(name_func, V14:V181)

head(bikes)

### Descriptive statistics

In [ ]:
plotcolors <- c(brewer.pal(name='Dark2', 8)[-1])
options(repr.plot.width = 12, repr.plot.height = 6)

plot_station <- function(stat,i) {
    ggplot() + geom_line(aes(c(0:167), stat[i, ]), color=plotcolors[i]) + labs(title=rownames(stat)[i]) +
    scale_y_continuous(name='Loading', limits=c(0, 1)) + 
    scale_x_continuous(name='Time', limits=c(0, 168))
}

options(repr.plot.width = 12, repr.plot.height = 6)
p1 <- plot_station(x,1)
p2 <- plot_station(x,2)
p3 <- plot_station(x,3)
p4 <- plot_station(x,4)
p5 <- plot_station(x,5)
p6 <- plot_station(x,6)

grid.arrange(p1,p2,p3,p4,p5,p6, nrow=2, ncol=3) 

In [ ]:
days = c('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday')

bikes_long <- bikes %>% pivot_longer(cols=T1:T168, names_to='time', values_to ='loading') %>% 
    mutate(time=as.numeric(substring(time, first=2))-1)

bikes.load_by_day <- bikes_long %>% 
    mutate(day=factor(days[(time %/% 24) + 1], ordered = TRUE, levels=days)) %>%
    mutate(hour=time %% 24) %>%
    group_by(day, hour) %>%
    summarise(mean_load=mean(loading)) %>%
    mutate(usage=100*(1-mean_load)) 

#bikes.load_by_day <- bikes.load_by_day %>% union(bikes.load_by_day %>%
#              filter(hour==0) %>%
#              mutate(hour=24) %>% 
#              ungroup() %>%
#              select(-day) %>%
#              bind_cols(tibble(day=factor(days[c(7, 1:6)], ordered = TRUE, levels=days))))
head(bikes.load_by_day)

In [ ]:
fig(9,5)
daycolors <- c(brewer.pal(name='YlGn', 6)[-1], brewer.pal(name='Blues', 9)[c(5,8)])

ggplot(bikes.load_by_day, aes(hour, mean_load, col=day)) + 
    geom_line() +
    geom_point() +
    scale_x_continuous(breaks = seq(0, 24, by=3)) +
    #scale_y_continuous(limits=c(55,70), breaks = seq(55, 70, by=5), labels=paste(seq(55, 70, by=5), '%', sep='')) +
    scale_y_continuous(limits=c(.3,.45), breaks = seq(.3, .45, by=.05)) +
    scale_color_manual(labels=days, values=daycolors) +
    #labs(title='Average loading by time of day', x='hour', y='average loading', color='Day')
    labs(x='Hour of day', y='Loading', color='Day') + 
    theme(text = element_text(size = 16))  

In [ ]:
bikes.load_by_hour <- bikes_long %>%
    filter(time < 5*24) %>%
    mutate(hour=time %% 24) %>%
    group_by(hour, name) %>%
    summarise(mean_load=mean(loading)) %>%
    inner_join(bikes %>% select(name, long, lat), by='name')

fig(12,5)

q1 <- qmplot(long, lat, data=bikes.load_by_hour %>% filter(hour==6), col=mean_load) +
    scale_colour_gradient(low='white', high='red', limits=c(0,1)) +
    labs(title='Average weekday loading at 06') +
    guides(color='none')

q2 <- qmplot(long, lat, data=bikes.load_by_hour %>% filter(hour==12), col=mean_load) +
    scale_colour_gradient(low='white', high='red', limits=c(0,1)) +
    labs(title='Average weekday loading at 12') +
    guides(color='none')

q3 <- qmplot(long, lat, data=bikes.load_by_hour %>% filter(hour==23), col=mean_load) +
    scale_colour_gradient(low='white', high='red', limits=c(0,1)) +
    labs(title='Average weekday loading at 23') +
    guides(color='none')

grid.arrange(q1, q2, q3, ncol=3)

In [ ]:
library(reshape2)
cormat <- cor(bikes %>% select(T1:T168))
melted_cormat <- melt(cormat)
ggplot(melted_cormat, aes(Var2, Var1, fill=value)) +
    scale_x_discrete(breaks=NULL, labels=NULL) + scale_y_discrete(breaks=NULL, limits=rev) +
    geom_raster() +
    scale_fill_gradient2("Correlation",low = "blue", mid="white", high = "red", midpoint=0, limits=c(-1,1)) +
    theme(axis.text.x = element_text(angle = 45, vjust = 1, size = 12, hjust = 1)) +
    coord_fixed()

In [ ]:
cormat <- cor(bikes %>% select(T1:T168))
avg.cormat <- matrix(0L, nrow=24, ncol=24)
for (i in 0:4) {
    for (j in 0:4) {
        avg.cormat <- avg.cormat + cormat[(24*i+1) : (24*(i+1)), (24*j+1) : (24*(j+1))]
    }
}
avg.cormat <- avg.cormat / 25
min(avg.cormat)
max(avg.cormat)
melted_cormat <- melt(avg.cormat)

fig(10,6)
ggplot(melted_cormat, aes(Var1, Var2, fill=value)) + 
    scale_x_discrete(position = 'top', labels=paste(c(0:23), 'h', sep=''), name='') +
    scale_y_discrete(limits=rev, labels=paste(c(23:0), 'h', sep=''), name='') +
    geom_raster() +
    scale_fill_gradient2("Correlation",low = "blue", mid="white", high = "red", midpoint=0, limits=c(-1,1)) +
    theme(legend.key.height=unit(.1, 'npc'),  text=element_text(size = 14)) +
    coord_fixed()

### Principal component analysis

In [ ]:
library(FactoMineR)
library(factoextra)

In [ ]:
num.pc <- 10
res.pca <- PCA(bikes %>% select(T1:T168), ncp=num.pc)

In [ ]:
fig(7,5)
p1 <- fviz_screeplot(res.pca, ncp=num.pc, addlabels=TRUE) + 
    scale_x_discrete(breaks=c(1:num.pc), labels=paste('PC', c(1:num.pc), sep='')) +
    labs(x='', y='Percentage of variance', title='Percentage of variance by principal component') +
    coord_cartesian(xlim=c(0.5, num.pc+0.5), ylim=c(0,41)) +
    theme_gray()

p2 <- ggplot() + geom_bar(stat="identity",aes(x=c(1:num.pc), y=res.pca$eig[1:num.pc,3]), fill='#4984B3') +
    labs(x='', y='Percentage of variance', title='Cumulative percentage of variance') +
    scale_x_continuous(breaks = c(1:num.pc), labels=paste('PC', c(1:num.pc), sep='')) + 
    scale_y_continuous(breaks = 25*c(0:4)) +
    annotate('segment', x=0, xend=num.pc+1, y=75, yend=75, col='black', linetype=2, size=0.5) +
    coord_cartesian(xlim=c(0.5, num.pc+0.5))

fig(10,4)
grid.arrange(p1, p2, nrow=1)

In [ ]:
plot_pc <- function(pc, i, label.days=TRUE) {
    day.labs <- rep('', 15)
    day.labs[2*c(1:7)] <- days
    g <- ggplot() + geom_line(aes(c(0:167), pc[,i])) + labs(y='', title=sprintf('PC%d loading profile', i)) + 
        scale_y_continuous(name='', limits=c(-1, 1)) + 
        #scale_x_continuous(name='', limits=c(0, 168), breaks=12*c(0:14), labels=NULL) +
        coord_cartesian(xlim=c(7, 161), ylim=c(-1,1)) + theme_minimal() +
        annotate('rect', 
                 xmin = 24*c(0:6), xmax = 24*c(1:7), 
                 ymin = rep(-1,7), ymax = rep(1,7), 
                 fill = daycolors, alpha = .2)
    if (label.days) {
        g <- g + scale_x_continuous(name='', limits=c(0, 168), breaks=12*c(0:14), labels=day.labs)
    } else {
        g <- g + scale_x_continuous(name='', limits=c(0, 168), minor_breaks=6*c(0:28), breaks=NULL)
    }
    g
}

In [ ]:
fig(12,10)
p1 <- plot_pc(res.pca$var$coord, 1)
p2 <- plot_pc(res.pca$var$coord, 2)
p3 <- plot_pc(res.pca$var$coord, 3)
p4 <- plot_pc(res.pca$var$coord, 4)
p5 <- plot_pc(res.pca$var$coord, 5)
grid.arrange(p1,p2,p3,p4,p5, ncol=1)

In [ ]:
bikes.pca <- as_tibble(res.pca$ind$coord) %>%
    select(num_range('Dim.', 1:5)) %>%
    rename_at(vars(paste('Dim.', c(1:5), sep='')), ~ paste('PC', c(1:5), sep='')) %>%
    bind_cols(bikes %>% select(name, long, lat, hill))

In [ ]:
plotPCA <- function(pca.data, color, ...) {
    g <- ggplot(pca.data, aes(color=as.factor(color))) +
        guides(color='none') +
        scale_x_continuous(position='top')

    p12 <- g + geom_point(aes(PC1, PC2), ...) +
        labs(x='PC1', y='PC2')
    p13 <- g + geom_point(aes(PC1, PC3), ...) +
        labs(x='', y='PC3')
    p14 <- g + geom_point(aes(PC1, PC4), ...) +
        labs(x='', y='PC4')
    p23 <- g + geom_point(aes(PC2, PC3), ...) +
        labs(x='PC2', y='')
    p24 <- g + geom_point(aes(PC2, PC4), ...) +
        labs(x='', y='')
    p34 <- g + geom_point(aes(PC3, PC4), ...) +
        labs(x='PC3', y='')
    
    lay <- rbind(c(1,NA,NA),
                 c(2, 4,NA),
                 c(3, 5, 6))
    grid.arrange(p12, p13, p14, p23, p24, p34, layout_matrix = lay)
}

In [ ]:
fig(10,10)
plotPCA(bikes.pca, bikes.pca$hill, alpha=.3, size=1, stroke=1)

In [ ]:
color_grad <- scale_color_gradient2("Dim1", low="blue", mid="white", high="red", midpoint=0)
t <- c(1:168)

gg.pc1 <- qmplot(long, lat, data=bikes.pca, col=PC1) +
    color_grad + 
    guides(color='none') + 
    labs(title='PC1') +
    theme(plot.margin=unit(c(0,0,0,0), 'in'))

gg.pc2 <- qmplot(long, lat, data=bikes.pca, col=PC2) +
    color_grad + 
    guides(color='none') + 
    labs(title='PC2') #+
    #theme(plot.margin=unit(c(0,1,0,0), 'npc'))

gg.profile <- ggplot() + 
    scale_x_continuous(name='', breaks=NULL, minor_breaks=24*c(0:7)) +
    scale_y_continuous(name='', limits=c(-1, 1), breaks=c(-1,0,1), minor_breaks=seq(-1,1, by=.5)) +
    coord_cartesian(xlim=c(7, 161), ylim=c(-1, 1)) +
    theme(plot.margin=unit(c(0,0,0,0), 'in'))

prof1 <- gg.profile + 
    geom_line(aes(t, res.pca$var$coord[,1])) + theme_minimal()

prof2 <- gg.profile + 
    geom_line(aes(t, res.pca$var$coord[,2])) + theme_minimal()

prof.height <- 1.6
fig(8,4)
gA1 <- ggplotGrob(gg.pc1)
gB1 <- ggplotGrob(prof1)
#gB1$widths[2:5] <- as.list(maxWidth)
gB1$heights[7] <- unit(prof.height, 'cm')

gA2 <- ggplotGrob(gg.pc2)
gB2 <- ggplotGrob(prof2)
#gB2$widths[2:5] <- as.list(maxWidth)
gB2$heights[7] <- unit(prof.height, 'cm')


grid::grid.newpage()
grid.arrange(rbind(gA1, gB1), rbind(gA2, gB2), ncol=2)

In [ ]:
#minlim <- min(bikes.pca$PC1, bikes.pca$PC2)
#maxlim <- max(bikes.pca$PC1, bikes.pca$PC2)
minlim <- -16
maxlim <- 16
color_grad <- scale_color_gradient2("PC coordinate", low="blue", mid="white", high="red", 
                                    midpoint=0, limits=c(minlim,maxlim), breaks=5*c(-3:3), oob=scales::squish)
t <- c(1:168)

gg.pc1 <- qmplot(long, lat, data=bikes.pca, col=PC1, shape=hill) +
    color_grad + 
    guides(color='none', shape='none') + 
    labs(title='PC1') +
    theme(plot.margin=unit(c(0,0,0,0), 'in'))

gg.pc2 <- qmplot(long, lat, data=bikes.pca, col=PC2) +
    color_grad + 
    labs(title='PC2') +
    theme(plot.margin=unit(c(0,.5,0,0), 'npc'), 
          legend.position=unit(c(1.2,.5), 'npc'),
          #legend.box.spacing=unit(c(0,0,0,0), 'npc'),
          #legend.key.size=unit(c(.01,.05), 'npc'),
          legend.key.height=unit(.06, 'npc'))


prof.height <- 1.6
fig(14,3)

lay <- rbind(c(1,2,2),
             c(1,2,2))
grid.arrange(gg.pc1, gg.pc2, layout_matrix=lay)

# K-means clustering

In [ ]:
max_k <- 10
loss <- vector(length=max_k)
clust.data <- bikes.pca %>% select(PC1:PC5)
for (k in 2:max_k) {
    clust <- kmeans(bikes %>% select(T1:T168), k)
    loss[k] = clust$tot.withinss
}
loss[1] = clust$totss
fig(7,5)
ggplot() + 
    geom_bar(stat="identity",aes(x=c(1:max_k), y=loss), fill='#4984B3') +
    geom_line(aes(x=c(1:max_k), y=loss)) +
    geom_point(aes(x=c(1:max_k), y=loss)) +
    scale_x_continuous(breaks = c(1:max_k), name = 'Number of clusters') +
    scale_y_continuous(breaks = seq(0, loss[1], length.out=5), 
                       labels = seq(0, 1, length.out=5), 
                       name = 'Within-cluster variance') +
    labs(title='Within-cluster variance relative to total variance') +
    theme(text = element_text(size = 14))

In [ ]:
permute_clustering <- function(km.mod) {
    freq <- table(km.mod$cluster)
    argsort <- order(freq)
    k <- length(argsort)
    km.mod$centers <- km.mod$centers[argsort,]
    km.mod$cluster <- factor(km.mod$cluster, levels=c(1:k), labels=argsort)
    km.mod
}

In [ ]:
K <- 5
p <- 5
pca.var_coord <- as.matrix(res.pca$var$coord[,1:5])
res.km5 <- kmeans(clust.data, 5, nstart=10)
km5centers <- matrix(nrow = 168, ncol = 5)
for (i in 1:5){
    km5centers[, i] <- res.pca$call$centre +
    res.pca$call$ecart.type * res.pca$var$coord[, 1:p] %*% (as.matrix(res.km5$centers[i, ], ncol = 1) / sqrt(res.pca$eig[1:p, 1])) # coord. in the orig. space
}

In [ ]:
plot_center <- function(centers, i) {
    day_lab <- rep('', 15)
    day_lab[2*c(1:7)] <- days
    t <- c(1:168)
    ggplot() + geom_line(aes(t, centers[,i])) + labs(y='', title=sprintf('Centroid %d loading profile', i)) + 
        scale_y_continuous(name='', limits=c(0, 1)) + 
        scale_x_continuous(name='', limits=c(0, 168), breaks=12*c(0:14), labels=day_lab) +
        coord_cartesian(xlim=c(7, 161), ylim=c(0,1)) +
        annotate('rect', xmin = 24*c(0:6), xmax = 24*c(1:7), ymin = rep(0,7), ymax = rep(1,7), 
                 fill=daycolors, alpha = .3) + theme_minimal() + theme(text = element_text(size = 12))
        
}

fig(8,8)
p1 <- plot_center(km5centers, 1)
p2 <- plot_center(km5centers, 2)
p3 <- plot_center(km5centers, 3)
p4 <- plot_center(km5centers, 4)
p5 <- plot_center(km5centers, 5)
grid.arrange(p1,p2,p3,p4,p5, ncol=1)

In [ ]:
K <- 7
p <- 5
pca.var_coord <- as.matrix(res.pca$var$coord[,1:5])
res.km7 <- kmeans(clust.data, K, nstart=10)
km7centers <- matrix(nrow = 168, ncol = K)
for (i in 1:K){
    km7centers[, i] <- res.pca$call$centre +
    res.pca$call$ecart.type * res.pca$var$coord[, 1:p] %*% 
    (as.matrix(res.km7$centers[i, ], ncol = 1) / sqrt(res.pca$eig[1:p, 1])) # coord. in the orig. space
}

fig(8,10)
p1 <- plot_center(km7centers, 1)
p2 <- plot_center(km7centers, 2)
p3 <- plot_center(km7centers, 3)
p4 <- plot_center(km7centers, 4)
p5 <- plot_center(km7centers, 5)
p6 <- plot_center(km7centers, 6)
p7 <- plot_center(km7centers, 7)
grid.arrange(p1,p2,p3,p4,p5,p6,p7, ncol=1)

In [ ]:
compare_full_v_pca <- function(centers.full, centers.pca, i) {
    days = c('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday')
    day_lab <- rep('', 15)
    day_lab[2*c(1:7)] <- days
    daycolors <- c(brewer.pal(name='YlGn', 6)[-1], brewer.pal(name='Blues', 9)[c(5,8)])
    t <- c(1:168)
    ggplot() + 
        geom_line(aes(t, centers.full[i,])) + labs(y='') +
        geom_line(aes(t, centers.pca[,i]), color='red') + 
        scale_y_continuous(name='', limits=c(0, 1)) + 
        scale_x_continuous(name='', limits=c(0, 168), breaks=12*c(0:14), labels=day_lab) +
        coord_cartesian(xlim=c(7, 161), ylim=c(0,1)) +
        annotate('rect', xmin = 24*c(0:6), xmax = 24*c(1:7), ymin = rep(0,7), ymax = rep(1,7), 
                 fill=daycolors, alpha = .2) + theme(text=element_text(size=14))
        
}


K <- 4
p <- 5
km.full <- kmeans(bikes %>% select(T1:T168), K)
km.full <- permute_clustering(km.full)
km.pca <- kmeans(bikes.pca %>% select(PC1:PC5), K)
km.pca <- permute_clustering(km.pca)

permut <- c(4,1,3,2)
km.full$centers <- km.full$centers[permut,]
km.full$cluster <- factor(km.full$cluster, labels=c(1:K), levels=permut)

km.pca$centers <- km.pca$centers[permut,]
km.pca$cluster <- factor(km.pca$cluster, levels=c(1:K), labels=permut)


kmcenters <- matrix(nrow = 168, ncol = K)
for (i in 1:K){
    kmcenters[, i] <- res.pca$call$centre +
    res.pca$call$ecart.type * res.pca$var$coord[, 1:p] %*% (as.matrix(km.pca$centers[i, ], ncol = 1) / sqrt(res.pca$eig[1:p, 1])) # coord. in the orig. space
}


fig(12,10)
p1 <- compare_full_v_pca(km.full$centers, kmcenters, 1) + labs(title='Centroid 1 loading profile')
p2 <- compare_full_v_pca(km.full$centers, kmcenters, 2) + labs(title='Centroid 2 loading profile')
p3 <- compare_full_v_pca(km.full$centers, kmcenters, 3) + labs(title='Centroid 3 loading profile')
p4 <- compare_full_v_pca(km.full$centers, kmcenters, 4) + labs(title='Centroid 4 loading profile')
grid.arrange(p1,p2,p3,p4, ncol=1)

In [ ]:
library(caret)
confusionMatrix(km.pca$cluster, factor(km.full$cluster, labels=c(1:4), levels=c(3,1,2,4)), dnn = c("PCA clustering", "Raw data clustering"))

In [ ]:
fig(10,7)
p1 <- plot_center(t(km.full$centers), 1) + labs(title='Centroid 1 loading profile [Low]') + theme_wsj()
p2 <- plot_center(t(km.full$centers), 2) + labs(title='Centroid 2 loading profile [High]') + theme_wsj()
p3 <- plot_center(t(km.full$centers), 3) + labs(title='Centroid 3 loading profile [Day]') + theme_wsj()
p4 <- plot_center(t(km.full$centers), 4) + labs(title='Centroid 4 loading profile [Night]') + theme_wsj()
grid.arrange(p1,p2,p3,p4, ncol=1)

In [ ]:
library(lsr)
gg_color_hue <- function(n) {
  hues = seq(15, 375, length = n + 1)
  hcl(h = hues, l = 65, c = 100)[1:n]
}
colors <- gg_color_hue(4)

fig(6.5,5)
ggplot(bikes.pca, aes(col=permuteLevels(km.full$cluster, c(4,3,2,1))  )) + 
    geom_point(aes(PC1, PC2)) + 
    theme(text=element_text(size=16)) +
    scale_color_discrete(name='Cluster', labels=c('1 [Low]', '2 [High]', '3 [Day]', '4 [Night]')) +
    annotate('point', x=km.pca$centers[,1], y=km.pca$centers[,2], shape=4, size=3, stroke=3)

In [ ]:
fig(6,5)
qmplot(long, lat, data=bikes.pca, color=permuteLevels(km.full$cluster, c(4,3,2,1)), shape=hill) + 
    guides(shape='none') +
    theme(text=element_text(size=16)) +
    scale_color_discrete(name='Cluster', labels=c('1 [Low]', '2 [High]', '3 [Day]', '4 [Night]'))

In [ ]:
plotPCAclustering <- function(pca.data, clusters, ...) {
    g <- ggplot(pca.data, aes(col=as.factor(clusters))) +
        guides(color='none') +
        scale_x_continuous(position='top')

    p12 <- g + geom_point(aes(PC1, PC2), ...) +
        labs(x='PC1', y='PC2')
    p13 <- g + geom_point(aes(PC1, PC3), ...) +
        labs(x='', y='PC3')
    p14 <- g + geom_point(aes(PC1, PC4), ...) +
        labs(x='', y='PC4')
    p23 <- g + geom_point(aes(PC2, PC3), ...) +
        labs(x='PC2', y='')
    p24 <- g + geom_point(aes(PC2, PC4), ...) +
        labs(x='', y='')
    p34 <- g + geom_point(aes(PC3, PC4), ...) +
        labs(x='PC3', y='')
    
    lay <- rbind(c(1,NA,NA),
                 c(2, 4,NA),
                 c(3, 5, 6))
    grid.arrange(p12, p13, p14, p23, p24, p34, layout_matrix = lay)
}

plotPCAclustering2 <- function(pca.data, clusters, ...) {
    g <- ggplot(pca.data, aes(col=as.factor(clusters))) +
        guides(color='none') +
        scale_x_continuous(position='top') +
        labs(x='', y='') +
        theme(plot.margin = unit(c(0,0,0,0), 'npc'))

    p12 <- g + geom_point(aes(PC2, PC1), ...) #+
        #labs(x='PC2', y='PC2')
    p13 <- g + geom_point(aes(PC1, PC3), ...) #+
        #labs(x='', y='PC3')
    p14 <- g + geom_point(aes(PC1, PC4), ...) #+
        #labs(x='', y='PC4')
    p23 <- g + geom_point(aes(PC2, PC3), ...) #+
        #labs(x='PC2', y='')
    p24 <- g + geom_point(aes(PC2, PC4), ...) #+
        #labs(x='', y='')
    p34 <- g + geom_point(aes(PC3, PC4), ...) #+
        #labs(x='PC3', y='')
    
    gg.text <- ggplot() + theme_bw() + 
        scale_x_continuous(breaks=NULL, name='') + 
        scale_y_continuous(breaks=NULL, name='') +
        theme(plot.margin = unit(c(0,0,0,0), 'npc'))
    t1 <- gg.text + annotate('text', 0,0, label='PC1')
    t2 <- gg.text + annotate('text', 0,0, label='PC2')
    t3 <- gg.text + annotate('text', 0,0, label='PC3')
    t4 <- gg.text + annotate('text', 0,0, label='PC4')
    
    lay <- rbind(c(7,1, 2, 3),
                 c(NA,8, 4, 5),
                 c(NA, NA,9, 6),
                 c(NA, NA, NA,10))
    grid.arrange(p12, p13, p14, p23, p24, p34,t1,t2,t3,t4, layout_matrix = lay)
}

In [ ]:
fig(10,10)
plotPCAclustering(bikes.pca, km.pca$cluster)

# Herarchical clustering

In [ ]:
D <- dist(bikes.pca %>% select(PC1:PC5))
bikes.hc <- hclust(D, 'ward.D2')

In [ ]:
require("cluster")
#sil <- silhouette(cutree(bikes.hc, 4), D)
#sil2 <- fviz_silhouette(silhouette(cutree(bikes.hc, 2), D))# + scale_y_discrete(labels=NULL) + coord_flip()
sil3 <- fviz_silhouette(silhouette(cutree(bikes.hc, 4), D))
sil4 <- fviz_silhouette(silhouette(cutree(bikes.hc, 4), D))
sil5 <- fviz_silhouette(silhouette(cutree(bikes.hc, 5), D))
sil6 <- fviz_silhouette(silhouette(cutree(bikes.hc, 6), D))
sil7 <- fviz_silhouette(silhouette(cutree(bikes.hc, 7), D))
sil8 <- fviz_silhouette(silhouette(cutree(bikes.hc, 8), D))
grid.arrange(sil3,sil4,sil5,sil6,sil7,sil8, ncol=2)

In [ ]:
hc.clusters <- cutree(bikes.hc, 5)
fig(10,10)
plotPCAclustering(bikes.pca, hc.clusters)

In [ ]:
fig(6,4)
qmplot(long, lat, data=bikes.pca, color=as.factor(hc.clusters)) + 
    #guides(shape='none') +
    theme(text=element_text(size=16)) +
    scale_color_discrete(name='Cluster', labels=c('1', '2', '3', '4', '5'))

In [ ]:
fig(10,7)
bikes %>%
    mutate(cluster = hc.clusters) %>%
    group_by(cluster) %>%
    select(c(cluster, T1:T168)) %>%
    summarise_each(funs(mean)) %>%
    pivot_longer(cols=T1:T168, names_to='time', values_to ='loading') %>%
    mutate(time=as.numeric(substring(time, first=2))-1) %>%
ggplot(aes(x=time, y=loading)) + geom_line() + facet_grid(cluster ~ .)

# Gaussian mixture model

In [ ]:
library(mclust)
library(plotmm)
library(mixtools)

In [ ]:
citation("mclust")

In [ ]:
fig(8,6)
mod.gm = Mclust(bikes.pca %>% select(PC1:PC5), G=c(5:20))
plot(mod.gm, what='BIC')

In [ ]:
summary(mod.gm)

In [ ]:
fig(10,10)
plotPCAclustering(bikes.pca, mod.gm$classification)

In [ ]:
gg_color_hue <- function(n) {
  hues = seq(15, 375, length = n + 1)
  hcl(h = hues, l = 65, c = 100)[1:n]
}
colors <- gg_color_hue(7)

fig(6,4.5)
ggplot(mutate(bikes.pca, cluster=as.factor(mod.gm$classification)), aes(color=cluster, shape=cluster )) + 
    geom_point(aes(PC1, PC2)) + 
    theme(text=element_text(size=16)) +
    scale_colour_manual(name = "Cluster",
                      labels = c("1", "2", "3", "4", "5", "6", "7"),
                      values = colors[c(6,7,1:5)]) +   
    scale_shape_manual(name = "Cluster",
                     labels = c("1", "2", "3", "4", "5", "6", "7"),
                     values = c(15:19, 7, 4))

In [ ]:

gg_color_hue <- function(n) {
  hues = seq(15, 375, length = n + 1)
  hcl(h = hues, l = 65, c = 100)[1:n]
}
colors <- gg_color_hue(10)



fig(8,6)
ggplot(mutate(bikes.pca, cluster=as.factor(mod.gm$classification)), aes(color=cluster, shape=cluster )) + 
    geom_point(aes(PC1, PC2)) + 
    theme(text=element_text(size=16)) +
    scale_colour_manual(name = "Cluster",
                      labels = c("1", "2", "3", "4", "5", "6", "7", "8", "9", "10"),
                      values = colors) +   
    scale_shape_manual(name = "Cluster",
                     labels = c("1", "2", "3", "4", "5", "6", "7", "8", "9", "10"),
                     values = c(8:18))

In [ ]:
fig(6,4)

colors <- gg_color_hue(7)

qmplot(long, lat, data=mutate(bikes.pca, cluster=as.factor(mod.gm$classification)), color=cluster, shape=cluster)+ 
    #guides(shape='none') +
  scale_colour_manual(name = "Cluster",
                      labels = c("1", "2", "3", "4", "5", "6", "7"),
                      values = colors[c(6,7,1:5)]) +   
  scale_shape_manual(name = "Cluster",
                     labels = c("1", "2", "3", "4", "5", "6", "7"),
                     values = c(15:19, 7, 4))

In [ ]:
fig(10,6)
day_lab <- rep('', 15)
day_lab[2*c(1:7)] <- days

bikes.box <- bikes %>% 
    select(T1:T168) %>%
    mutate(cluster = as.factor(mod.gm$classification)) %>%
    filter(cluster %in% c(4,7)) %>%
    pivot_longer(cols=T1:T168, names_to='time', values_to ='loading') %>%
    mutate(time=factor(as.numeric(substring(time, first=2))-1, ordered=TRUE))

b4 <- ggplot(bikes.box %>% filter(cluster==4), aes(x=time, y=loading)) +
    annotate('rect', 
                 xmin = 24*c(0:6), xmax = 24*c(1:7), 
                 ymin = rep(0,7), ymax = rep(1,7), 
                 fill = daycolors, alpha = .3) + 
    geom_boxplot(fill='gray') + 
    scale_x_discrete(breaks=12*c(0:14), labels=day_lab) + 
    labs(x='', y='', title='Boxplot of station loading in cluster 4') + 
    theme(text = element_text(size = 14))  

b7 <- ggplot(bikes.box %>% filter(cluster==7), aes(x=time, y=loading)) +
    annotate('rect', 
                 xmin = 24*c(0:6), xmax = 24*c(1:7), 
                 ymin = rep(0,7), ymax = rep(1,7), 
                 fill = daycolors, alpha = .3) + 
    geom_boxplot(fill='gray') + 
    scale_x_discrete(breaks=12*c(0:14), labels=day_lab) + 
    labs(x='', y='', title='Boxplot of station loading in cluster 7') + 
    theme(text = element_text(size = 14))  

grid.arrange(b4, b7, ncol=1)

In [ ]:
library(ggmosaic)
ggplot(tibble(hill=as.factor(bikes$hill), cluster=as.factor(mod.gm$classification))) +
  geom_mosaic(aes(x = product(hill, cluster)))

In [ ]:
K <- 7
p <- 5
pca.var_coord <- as.matrix(res.pca$var$coord[,1:5])
res.gmm <- Mclust(bikes.pca %>% select(PC1:PC5), G=K, model='VVE')
centers <- matrix(nrow = 168, ncol = K)
for (i in 1:K){
    centers[, i] <- res.pca$call$centre +
    res.pca$call$ecart.type * res.pca$var$coord[, 1:p] %*% 
    (as.matrix(res.gmm$parameters$mean[,i], ncol = 1) / sqrt(res.pca$eig[1:p, 1])) # coord. in the orig. space
}

fig(8,10)
p1 <- plot_center(centers, 1)
p2 <- plot_center(centers, 2)
p3 <- plot_center(centers, 3)
p4 <- plot_center(centers, 4)
p5 <- plot_center(centers, 5)
p6 <- plot_center(centers, 6)
p7 <- plot_center(centers, 7)
grid.arrange(p1,p2,p3,p4,p5,p6,p7, ncol=1)